In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\project Series\\Chest_Cancer_Classification_DL\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\project Series\\Chest_Cancer_Classification_DL'

In [5]:
from  pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int
    params_weights: str
    params_include_top: bool

In [6]:
from Cnnclassifier.utils.common import read_yaml, create_directories
from Cnnclassifier.constants import *

In [7]:
class ConfigManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        # Read config and params YAML
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create artifacts root directory
        create_directories([self.config.artifacts_root])

    def get_base_model_config(self) -> BaseModelConfig:
        base_model_config = self.config.base_model
        
        return BaseModelConfig(
            
            root_dir=Path(base_model_config.root_dir),
            base_model_path=Path(base_model_config.base_model_path),
            updated_base_model_path=Path(base_model_config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.classes,
            params_weights=self.params.WEIGHTS,
            params_include_top=self.params.INCLUDE_TOP
        )


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

c:\Users\Qosain\anaconda3\envs\chest_cancer_dl\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [9]:
from Cnnclassifier import logger
from pathlib import Path
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    def get_base_model(self):
        base_model = tf.keras.applications.vgg16.VGG16(
            input_shape=tuple(self.config.params_image_size),
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,
        )

        # Ensure directory exists
        self.config.base_model_path.parent.mkdir(parents=True, exist_ok=True)

        base_model.save(self.config.base_model_path)
        logger.info(f"Base model saved at: {self.config.base_model_path}")

        return base_model

    @staticmethod
    def _prepare_full_model(model, classes, learning_rate, freeze_all, freeze_till):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
        else:
            for layer in model.layers:
                layer.trainable = True

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        # Prepare the updated model
        self.full_model = self._prepare_full_model(
            model=tf.keras.models.load_model(self.config.base_model_path),
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate,
            freeze_all=True,
            freeze_till=None
        )

        # Save the updated model
        self.save_model(self.full_model, self.config.updated_base_model_path)

        return self.full_model

    @staticmethod
    def save_model(model: tf.keras.Model, path: Path):
        path.parent.mkdir(parents=True, exist_ok=True)
        model.save(path)
        logger.info(f"Model saved at: {path}")


In [10]:
try:
    config=ConfigManager()
    PrepareBaseModelConfig=config.get_base_model_config()
    prepare_base_model=PrepareBaseModel(config=PrepareBaseModelConfig)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    logger.exception(e)
    raise e

[2025-12-18 23:16:16,195 : INFO : cnnclassifierlogger : yaml file: config\config.yaml loaded successfully]
[2025-12-18 23:16:16,198 : INFO : cnnclassifierlogger : yaml file: params.yaml loaded successfully]
[2025-12-18 23:16:16,199 : INFO : cnnclassifierlogger : Created directory at: artifacts]
[2025-12-18 23:16:16,565 : WARNING : absl : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-12-18 23:16:16,668 : INFO : cnnclassifierlogger : Base model saved at: artifacts\base_model\base_model.h5]
[2025-12-18 23:16:16,850 : WARNING : absl : No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

[2025-12-18 23:16:16,898 : WARNING : absl : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-12-18 23:16:17,031 : INFO : cnnclassifierlogger : Model saved at: artifacts\base_model\updated_base_model.h5]
